In [1]:
import pandas as pd
# machine-readable-puf.csv is converted from the excel file in http://download.cms.gov/marketplace-puf/2016/machine-readable-url-puf.zip
machine_readable_df = pd.read_csv('/home/xinlu/Downloads/hhs_csvs/machine-readable-puf.csv')
urls = machine_readable_df[machine_readable_df['URL Submitted'].str.startswith('http')]['URL Submitted'].unique()
print 'There are', len(urls), 'unique urls, from', machine_readable_df['Issuer ID'].nunique(),'issuers'

There are 149 unique urls, from 484 issuers


In [2]:
machine_readable_df.head(2)

,State,Issuer ID,Issuer Name,URL Submitted,Tech POC Email
0,AK,21989,Oregon Dental Service,https://www.modahealth.com/cms-data-index.json,jessica.wagner@modahealth.com
1,AK,38344,Premera Blue Cross Blue Shield of Alaska,https://fm.formularynavigator.com/jsonFiles/pu...,jim.tedford@premera.com


urls = machine_readable_df[(machine_readable_df.State=='HI') & 
                           machine_readable_df['URL Submitted'].str.startswith('http')]['URL Submitted'].unique()

# Download the json files

In [4]:
# data from each url will be saved separately under this folder
data_directory = './provider_json_files/'

In [5]:
import urllib2,json
from multiprocessing.dummy import Pool as ThreadPool
import sys
import os

In [13]:
def url_to_filename(url):
    return url.replace('http://','').replace('https://','').replace('/','-')

def get_filename(url):
    return url.split('/')[-1]

def download(url):    
    print url
    folder = url_to_filename(url)
    if not folder in os.listdir(data_directory): 
        os.makedirs(data_directory+folder)
    else:
        if sum(1 for d in os.listdir(data_directory+folder) if 'json' in d) > 2:
            print 'Downloaded'
            return
    
    #pool = ThreadPool(8) 
    res = json.load(urllib2.urlopen(url))
    
    def d(url):
        filename = get_filename(url)
        if filename in os.listdir(data_directory+folder): 
            print folder+filename, 'is already downloaded'
            return
        handle = urllib2.urlopen(url)
        if int(handle.headers['content-length']) > 1024*1024*50:
            print folder+'/'+filename, 'is larger than 50 MB (%.2f MB), skipping' % 1.*int(handle.headers['content-length'])/1024/1024
            return
        else:
            print folder+'/'+get_filename(url), round(1.*int(handle.headers['content-length'])/1024/1024,2), 'MB'
            
        with open(data_directory+folder+'/'+filename,'wb') as f:
            f.write(handle.read()) # TODO should be chunking the read/save 
    
    #pool.map(d, res['plan_urls'])
    map(d, res['plan_urls'])
    map(d, res['provider_urls'])
    return

pool = ThreadPool(8) 
#res = [(u, download(u)) for u in urls]
pool.map(download, urls[2:10])

http://www.bestlife.com/exchange/cms-data-index.json
https://api.humana.com/v1/cms/index.json
https://www.bcbsal.org/cms/cms-data-index.json
https://groupaccess.deltadentalil.com/cmsdata/index.json
https://unitedhealthcarecms.com/cms-data-index.json
http://www.dentegra.com/hcx/cms-data-index.json
https://www.deltadental.com/CMSDirectory/index.json
https://provider-search.qualchoice.com/index.json
Downloaded
www.bestlife.com-exchange-cms-data-index.jsonplans.json is already downloaded
provider-search.qualchoice.com-index.jsonplans.json is already downloaded
www.deltadental.com-CMSDirectory-index.jsonplans.json is already downloaded
groupaccess.deltadentalil.com-cmsdata-index.jsonplans.json is already downloaded
unitedhealthcarecms.com-cms-data-index.jsonJSON_PLANS_UHC.json is already downloaded
www.bestlife.com-exchange-cms-data-index.json/providers_wNPI.json provider-search.qualchoice.com-index.json/providers.json groupaccess.deltadentalil.com-cmsdata-index.json/providers.jsonwww.delta

# Parsing

In [6]:
files = []
for d in os.listdir(data_directory):
    for file in os.listdir(data_directory+d):
        print file
        if 'provider' in file.lower():
            files.append(data_directory+d+'/'+file)

plans.json
providers.json
plans.json
plans.json
providers.json
plans.json
providers.json
plans.json
JSON_PLANS_UHC.json
JSON_Providers_UHC_Vision.json
plans.json
providers_bcbsal.json
providers_platinum_gold.json
providers_silver_bronze.json
plans.json
providers_woNPI.json
plans.json
plans.json
providers.json
plans-AK.json
plans-OR.json


In [9]:
file = files[0]
with open(file) as f:
    providers = json.load(f)
    print providers[0]['plans'][0]
    
    del providers[0]['plans']
    
    print json.dumps(providers[0],indent=2)

{u'plan_id_type': u'HIOS-PLAN-ID', u'plan_id': u'38344AK0620003', u'network_tier': u'HERITAGE-PLUS'}
{
  "npi": "1063645026", 
  "facility_name": "EXPRESS SCRIPTS", 
  "facility_type": [
    "Pharmacies"
  ], 
  "last_updated_on": "2015-10-14", 
  "type": "FACILITY", 
  "addresses": [
    {
      "city": "WHITESTOWN", 
      "state": "IN", 
      "phone": "2012695236", 
      "zip": "46075", 
      "address": "4750 E 450 S"
    }
  ]
}


In [14]:
df = pd.read_json(files[0])
print df.shape
df.head(1)

(3326, 12)


,accepting,addresses,facility_name,facility_type,gender,languages,last_updated_on,name,npi,plans,speciality,type
0,NaN,"[{u'city': u'WHITESTOWN', u'state': u'IN', u'p...",EXPRESS SCRIPTS,[Pharmacies],NaN,NaN,2015-10-14,NaN,1063645026,"[{u'plan_id_type': u'HIOS-PLAN-ID', u'plan_id'...",NaN,FACILITY


## saving the mapping between npi and plan_id to hdf5 format for quickly querying

In [27]:
from pandas.io.json import json_normalize
df_tot = pd.DataFrame()
for file in files:
    print file
    df = json_normalize(json.load(open(file)),'plans', ['npi']).drop('plan_id_type',axis=1)
    df['npi'] = df['npi'].astype(str)
    df['network_tier'] = df['network_tier'].astype(str)
    df['plan_id'] = df['plan_id'].astype(str)
    df.set_index('npi')
    df_tot = pd.concat([df_tot,df])
    
df = df_tot
for state in df.plan_id.str.slice(5,7).unique():
    df[df.plan_id.str.slice(5,7)==state].to_hdf('plan_provider.h5',str(state),format='t',append=True)
    #df.head(1)

./provider_json_files/fm.formularynavigator.com-jsonFiles-publish-11-47-cms-data-index.json/providers.json
./provider_json_files/healthy.kaiserpermanente.org-static-health-json-technical_information-hi-cms-data-index.json/providers.json
./provider_json_files/hmsa.com-cms-data-index.json/providers.json
./provider_json_files/unitedhealthcarecms.com-cms-data-index.json/JSON_Providers_UHC_Vision.json
./provider_json_files/www.bcbsal.org-cms-cms-data-index.json/providers_bcbsal.json
./provider_json_files/www.bcbsal.org-cms-cms-data-index.json/providers_platinum_gold.json
./provider_json_files/www.bcbsal.org-cms-cms-data-index.json/providers_silver_bronze.json
./provider_json_files/www.bestlife.com-exchange-cms-data-index.json/providers_woNPI.json
./provider_json_files/www.dentegra.com-hcx-cms-data-index.json/providers.json


In [29]:
print len(pd.read_hdf('plan_provider.h5','HI'))

157573


## saving providers to hdf5

In [91]:
file = files[0]
df = pd.read_json(file)
df.drop([c for c in ['plans','last_updated_on','type','languages','gender','facility_type'] if c in df.columns],axis=1,)
    # normalize the address (keeping only the first one for now) and name
    
#df = pd.concat([df, df.pop('name').apply(pd.Series).drop([0],axis=1),df.pop('addresses').apply(pd.Series)[0].apply(pd.Series)], axis=1)
df.head(1)

,accepting,addresses,facility_name,facility_type,gender,languages,last_updated_on,name,npi,plans,speciality,type
0,NaN,"[{u'city': u'WHITESTOWN', u'state': u'IN', u'p...",EXPRESS SCRIPTS,[Pharmacies],NaN,NaN,2015-10-14,NaN,1063645026,"[{u'plan_id_type': u'HIOS-PLAN-ID', u'plan_id'...",NaN,FACILITY


In [117]:
import numpy as np
file = files[0]
init = False
df_tot = None
for file in files[:]:
    print file
    df = pd.read_json(file)
    df.drop([c for c in ['plans','last_updated_on','type','languages','gender','facility_type'] if c in df.columns],axis=1,)
    # normalize the address (keeping only the first one for now) and name
    if 'name' in df.columns:
        df = pd.concat([df, df.pop('name').apply(pd.Series)], axis=1)
        if 0 in df.columns: df.drop([0],axis=1)
    if 'addresses' in df.columns:
        df = pd.concat([df, df.pop('addresses').apply(pd.Series)[0].apply(pd.Series)], axis=1)
    if not init: 
        df_tot = df
        init = True
    else: df_tot = pd.concat([df_tot,df])
    for s in ['npi', u'accepting', 'address','address_2','city','phone','state','zip', u'facility_name', u'facility_type', u'gender', 
          u'languages', u'last_updated_on',  u'npi', u'plans', u'speciality', u'type', 'suffix', 'prefix','first','middle','last']:
        def fix(s):
            try:
                return str(s)
            except: return u''
        df[s] = df[s].fillna(u'').apply(fix).astype(str)
        
    #df.to_hdf('test.h5','test',format='t',append=True)
df_tot.to_hdf('test.h5','providers',format='t',append=False)

./provider_json_files/fm.formularynavigator.com-jsonFiles-publish-11-47-cms-data-index.json/providers.json
./provider_json_files/healthy.kaiserpermanente.org-static-health-json-technical_information-hi-cms-data-index.json/providers.json
./provider_json_files/hmsa.com-cms-data-index.json/providers.json
./provider_json_files/unitedhealthcarecms.com-cms-data-index.json/JSON_Providers_UHC_Vision.json
./provider_json_files/www.bcbsal.org-cms-cms-data-index.json/providers_bcbsal.json
./provider_json_files/www.bcbsal.org-cms-cms-data-index.json/providers_platinum_gold.json
./provider_json_files/www.bcbsal.org-cms-cms-data-index.json/providers_silver_bronze.json
./provider_json_files/www.bestlife.com-exchange-cms-data-index.json/providers_woNPI.json
./provider_json_files/www.dentegra.com-hcx-cms-data-index.json/providers.json


In [160]:
df1 = pd.read_hdf('test.h5','providers')
print df.shape, df['npi'].nunique()

(68852, 16) 51408
